In [1]:
import jieba
import os
import re
import numpy as np
import jieba.posseg as psg
import networkx as nx
import pandas as pd

In [2]:
# -*- coding:utf-8 -*-
import pymysql

# ******************************************数据库操作***************************************************#
class Database:
    # 数据库初始化，port得是数值int型,暂时先这样，（后续情况需要可以开放多库多服务器查询（允许传入参数）
    def __init__(self, ):
        self.host = '127.0.0.1'  # 主机名，默认本地
        # self.host = '101.201.198.35'  # 主机名，为服务器ip
        self.port = 3306  # 端口默认3306
        self.user = 'root'  # 用户名，默认root
        self.password = '13886139062wang'  # 密码
        # self.password = 'STourism@whu'  # 服务器密码
        self.db = 'zhihu'  # 数据库名
        self.charset = 'utf8'  # 数据库字符集，默认utf-8,7.31 xts,修改为utf8mb4，支持存储表情

    # **********************************数据库连接并执行的相关函数*********************************************#2.12更新可以批量插入和查询一个
    def database(self, sql, fetch='all', data_list='one'):
        o_type = sql.split(' ')[0].lower()
        conn = pymysql.connect(host=self.host, port=self.port, user=self.user, password=self.password, db=self.db, charset=self.charset)  # 连接数据库
        cur = conn.cursor()  # 用于访问和操作数据库中的数据（一个游标，像一个指针）
        if o_type == 'select':
            cur.execute(sql)  # 执行操作
            if fetch == 'all':
                result = cur.fetchall()  # 匹配所有满足的
            else:
                result = cur.fetchone()  # 匹配一个满足的
            cur.close()  # 关闭游标
            conn.close()  # 关闭数据库连接
            return result
        elif o_type == 'insert' or o_type == 'update' or o_type == 'delete':
            if data_list == 'one':
                try:
                    cur.execute(sql)
                    conn.commit()  # 提交事务
                    # print("{} ok".format(type))
                except Exception as e:  # 发生错误时回滚
                    print(e)
                    conn.rollback()  # 回滚事务
            else:  # 一次插入多条数据
                try:
                    cur.executemany(sql, data_list)  # 列表里面的单个数据必须是元组
                    conn.commit()  # 提交事务
                    # print("{} ok".format(type))
                except Exception as e:  # 发生错误时回滚
                    print(e)
                    conn.rollback()  # 回滚事务
            cur.close()  # 关闭游标
            conn.close()  # 关闭数据库连接


if __name__ == '__main__':
    Data_admin = Database()

In [3]:
def get_stop_dict(file):
    content = open(file,encoding="utf-8")
    word_list = []
    for c in content:
        c = re.sub('\n|\r','',c)
        word_list.append(c)
    return word_list

In [4]:
def get_data(path):
    t = open(path,encoding="utf-8")
    data = t.read()
    t.close()
    return data

In [5]:
def get_wordlist(text,maxn,synonym_words,stop_words):
    synonym_origin = list(synonym_words['origin'])
    synonym_new = list(synonym_words['new'])
    flag_list = ['n','nz','vn']#a,形容词，v,形容词
    counts={}
    
    new_text = ''

    for index in text.index:
        index_text = text.loc[index].values[0]
        new_text = new_text + index_text + '\n'

    
    text_seg = psg.cut(new_text)
    for word_flag in text_seg:
        #word = re.sub("[^\u4e00-\u9fa5]","",word_flag.word)
        word = word_flag.word
        if word_flag.flag in flag_list and len(word)>1 and word not in stop_words:
            if word in synonym_origin:
                index = synonym_origin.index(word)
                word = synonym_new[index]
            counts[word]=counts.get(word,0)+1
            
    
    words= sorted(counts.items(),key=lambda x:x[1],reverse=True)
    words= list(dict(words).keys())[0:maxn]
    
    return words

In [6]:
def get_t_seg(topwords,text,synonym_words,stop_words):
    synonym_origin = list(synonym_words['origin'])
    synonym_new = list(synonym_words['new'])
    flag_list = ['n','nz','vn']#a,形容词，v,形容词
    
    text_lines_seg =[]

    for index in text.index:
        t_seg = []
        text_seg = psg.cut(text.loc[index].values[0])
        for word_flag in text_seg:
            # word = re.sub("[^\u4e00-\u9fa5]","",word_flag.word)
            word = word_flag.word
            if word_flag.flag in flag_list and len(word)>1 and word not in stop_words:
                if word in synonym_origin:
                    word = synonym_new[synonym_origin.index(word)]
                if word in topwords:
                    t_seg.append(word)
        t_seg=list(set(t_seg))
        text_lines_seg.append(t_seg)
    return text_lines_seg

In [7]:
def get_comatrix(text_lines_seg):
    comatrix = pd.DataFrame(np.zeros([len(topwords),len(topwords)]),columns=topwords,index=topwords)
    for t_seg in text_lines_seg:
        for i in range(len(t_seg)-1):
                for j in range(i+1,len(t_seg)):
                    comatrix.loc[t_seg[i],t_seg[j]]+=1
                    comatrix.loc[t_seg[j],t_seg[i]]+=1
    for k in range(len(comatrix)):
        comatrix.iloc[k,k]=0
    return comatrix

In [8]:
def get_net(co_matrix,topwords):
    g = nx.Graph()
    for i in range(len(topwords)-1):
        word = topwords[i]
        for j in range(i+1,len(topwords)):
            w=0
            word2 = topwords[j]
            w = co_matrix.loc[word][word2]
            if w>0:
                g.add_edge(word,word2,weight=w)
    return g

In [9]:
#文件路径
dic_file = "../stop_dic/dict.txt"
stop_file = "../stop_dic/stopwords.txt"
# data_path = "../data/data.txt"
synonym_file = "../stop_dic/synonym_list.xlsx"

In [10]:
import pandas as pd

sql = 'select answer_text from zhihu_it_cleaned'
raw_Data = Data_admin.database(sql)
data = pd.DataFrame(raw_Data)

data

,0
0,这不很明显的诡辩、偷换概念吗？把公司的问题转移成社会问题。\n正常的逻辑难道不应该是：\n虽...
1,跟996福报有异曲同工之妙
2,支持并夕夕！\n你们看看种植园的黑奴，哪一个不是用命摘棉花，我一直不以为是奴隶主的问题，而是...
3,2015年4月，拼多多正式上线。仅用不到一年时间，在16年2月，拼多多单月成交额破1000万...
4,三句话：\n1.真相是杀不完的，因为真理永远存在\n2.吊死资本家的绳索，他们自己会给我们\...
...,...
40067,眼看热榜第四之后就消失了，我却什么事都做不了，深感抱歉。\n最近刚看完《悲惨世界》，里面女工...
40068,昨晚半夜看到这篇文章，很震惊也很痛心。一直认为网易是一家有温度有情怀的互联网大厂，做出这样无...
40069,网易有态度
40070,"想到了鲁迅先生的狂人日记，整个公司写满了""吃人""。"


In [11]:
#读取文件
# data = get_data(data_path)
stop_words = get_stop_dict(stop_file)
jieba.load_userdict(dic_file)
synonym_words = pd.read_excel(synonym_file)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/6d/2k7_g1z929x_ljmzs94gg_kw0000gn/T/jieba.cache
Loading model cost 0.481 seconds.
Prefix dict has been built successfully.


In [12]:
#数据处理
n_topwords = 200
topwords = get_wordlist(data,n_topwords,synonym_words,stop_words)
t_segs = get_t_seg(topwords,data,synonym_words,stop_words)

In [13]:
co_matrix = get_comatrix(t_segs)
co_net = get_net(co_matrix,topwords)
co_matrix

,公司,问题,员工,平台,时间,骑手,企业,工作,百度,网易,...,话题,运营,合理,坦克,方法,理由,单位,礼物,垃圾,研究
公司,0.0,1381.0,1598.0,475.0,932.0,243.0,1138.0,1308.0,383.0,924.0,...,117.0,185.0,167.0,9.0,164.0,214.0,128.0,32.0,121.0,154.0
问题,1381.0,0.0,886.0,1037.0,1281.0,700.0,958.0,923.0,345.0,423.0,...,198.0,164.0,221.0,87.0,209.0,193.0,122.0,72.0,101.0,156.0
员工,1598.0,886.0,0.0,285.0,635.0,111.0,869.0,1002.0,85.0,841.0,...,78.0,128.0,135.0,2.0,119.0,160.0,134.0,8.0,59.0,63.0
平台,475.0,1037.0,285.0,0.0,1156.0,1093.0,330.0,328.0,110.0,74.0,...,77.0,104.0,170.0,20.0,112.0,94.0,42.0,65.0,45.0,50.0
时间,932.0,1281.0,635.0,1156.0,0.0,952.0,621.0,833.0,128.0,261.0,...,101.0,118.0,259.0,21.0,158.0,160.0,99.0,44.0,59.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
理由,214.0,193.0,160.0,94.0,160.0,53.0,127.0,142.0,25.0,80.0,...,18.0,28.0,34.0,3.0,25.0,0.0,27.0,7.0,11.0,26.0
单位,128.0,122.0,134.0,42.0,99.0,24.0,102.0,135.0,11.0,72.0,...,9.0,10.0,20.0,1.0,22.0,27.0,0.0,2.0,8.0,12.0
礼物,32.0,72.0,8.0,65.0,44.0,4.0,8.0,24.0,7.0,2.0,...,24.0,16.0,10.0,40.0,12.0,7.0,2.0,0.0,5.0,7.0
垃圾,121.0,101.0,59.0,45.0,59.0,11.0,56.0,48.0,113.0,61.0,...,7.0,7.0,9.0,6.0,14.0,11.0,8.0,5.0,0.0,13.0


In [14]:
new_comatrix = pd.DataFrame.copy(co_matrix, deep=True)
for index in new_comatrix.index:
        new_comatrix.loc[new_comatrix[index]<200, index] = 0
new_conet = get_net(new_comatrix,topwords)

In [15]:
new_comatrix

,公司,问题,员工,平台,时间,骑手,企业,工作,百度,网易,...,话题,运营,合理,坦克,方法,理由,单位,礼物,垃圾,研究
公司,0.0,1381.0,1598.0,475.0,932.0,243.0,1138.0,1308.0,383.0,924.0,...,0.0,0.0,0.0,0.0,0.0,214.0,0.0,0.0,0.0,0.0
问题,1381.0,0.0,886.0,1037.0,1281.0,700.0,958.0,923.0,345.0,423.0,...,0.0,0.0,221.0,0.0,209.0,0.0,0.0,0.0,0.0,0.0
员工,1598.0,886.0,0.0,285.0,635.0,0.0,869.0,1002.0,0.0,841.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
平台,475.0,1037.0,285.0,0.0,1156.0,1093.0,330.0,328.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
时间,932.0,1281.0,635.0,1156.0,0.0,952.0,621.0,833.0,0.0,261.0,...,0.0,0.0,259.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
理由,214.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
单位,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
礼物,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
垃圾,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
nx.write_gexf(co_net,"../result/it.gexf")
nx.write_gexf(new_conet,"../result/new_it.gexf")